In [1]:
import numpy as np # to perform algebric calculations
import pandas as pd # data processing-input as pandas dataframe
import os
import gc # To clean memory
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.cross_validation import KFold
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings('ignore')

/home/nish_sharon/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
#import CSVs
clicks_train = pd.read_csv("/home/nisha/Outbrain/clicks_train.csv")

promoted_content = pd.read_csv("/home/nisha/Outbrain/promoted_content.csv")

clicks_test=pd.read_csv("/home/nisha/Outbrain/clicks_test.csv")

print("import completed")

import completed


In [3]:
#fill 0s for missing values
promoted_content.fillna(0, inplace=True)
clicks_train.fillna(0,inplace=True)

In [4]:
clicks_train.shape

(87141731, 3)

In [5]:
clicks_test.shape

(32225162, 2)

In [4]:
#merge training clicks with promoted content on ad id
train_promoted=clicks_train.merge(promoted_content,on='ad_id' ,how='left')

In [5]:
#calculate ads ctr 
train_ctr=clicks_train.groupby(['ad_id']).clicked.agg(['count' ,'sum']).reset_index()
train_ctr['prob']=train_ctr['sum']/train_ctr['count']
train_ctr=train_ctr.drop(['count','sum'],1)
print(train_ctr)

         ad_id      prob
0            1  0.000000
1            2  0.045455
2            3  0.068323
3            4  0.125000
4            5  0.000000
5            6  0.000000
6            7  0.289196
7            8  0.142857
8            9  0.000000
9           10  0.111111
10          11  0.000000
11          12  0.166667
12          13  0.125000
13          14  0.100000
14          15  0.000000
15          16  0.333333
16          17  0.100000
17          18  0.000000
18          20  0.000000
19          21  0.000000
20          22  0.000000
21          23  0.181818
22          24  0.000000
23          25  0.000000
24          26  0.045455
25          27  0.000000
26          28  0.000000
27          29  0.000000
28          30  0.000000
29          31  0.166667
...        ...       ...
478920  547501  0.000000
478921  547504  0.000000
478922  547505  0.000000
478923  547509  0.000000
478924  547513  0.000000
478925  547522  0.125000
478926  547548  0.750000
478927  547552  0.000000


In [6]:
#merge the calculated ctr to promoted_content based on 'ad_id'
promoted_content=promoted_content.merge(train_ctr,on='ad_id' ,how='left')
promoted_content.fillna(0, inplace=True)
del train_ctr

In [7]:
events = pd.read_csv("/home/nisha/Outbrain/events.csv")
events=events[['display_id','platform']]
events.platform = events.platform.astype(str)

In [9]:
train_events=clicks_train.merge(events,on='display_id' ,how='left')
train_events=train_events.groupby('platform').clicked.agg(['count' ,'sum']).reset_index()
train_events['platform_ctr']= (train_events['sum'] ) / (train_events['count'])

In [8]:
train_events=train_events.drop(['platform','count','sum'],1)
clicks_test=clicks_test.merge(events,on='display_id' ,how='left')

clicks_train+events done
clicks_test+events done


In [10]:
#Merge click_trains and promoted_content on ad_id and calculate probability
promoted_campaign= promoted_content.groupby('campaign_id').clicked.agg(['count' ,'sum']).reset_index()
promoted_campaign['cmpg_prob']= (promoted_adv['sum']) / (promoted_adv['count'])
promoted_campaign=pd.DataFrame(promoted_campaign).sort_values('cmpg_prob')
promoted_campaign=promoted_campaign.drop('count',1)
promoted_campaign=promoted_campaign.drop('sum',1)

In [11]:
#Merge click_trains and  promoted_content on ad_id and calculate advertiser_score
promoted_adv= promoted_content.groupby('advertiser_id').clicked.agg(['count' ,'sum']).reset_index()
promoted_adv['adv_prob']= (promoted_adv['sum']) / (promoted_adv['count'])
promoted_adv=promoted_adv.drop('count',1)
promoted_adv=promoted_adv.drop('sum',1)

In [12]:
#add the calculated campaigner_score and advertiser_score to promoted_content
promoted_content=promoted_content.merge(promoted_adv,on='advertiser_id',how='left').merge(df_cmpg,on='campaign_id',how='left')
promoted_content=promoted_content.drop('campaign_id',1)
promoted_content=promoted_content.drop('advertiser_id',1)

campg_score and adv_score done


In [13]:
print(promoted_content)

         ad_id  document_id  prob_Score  adv_Score  cmpg_Score
0            1         6614    0.000000   0.202470    0.240752
1            2       471467    0.025000   0.202470    0.125249
2            3         7692    0.061453   0.202470    0.113260
3            4       471471    0.080000   0.202470    0.125249
4            5       471472    0.000000   0.202470    0.125249
5            6        12736    0.000000   0.202470    0.240752
6            7        12808    0.287675   0.202470    0.240752
7            8       471477    0.086957   0.202470    0.125249
8            9        13379    0.000000   0.202470    0.240752
9           10        13885    0.090909   0.202470    0.240752
10          11        14230    0.000000   0.202470    0.240752
11          12       446701    0.066667   0.113761    0.067039
12          13       471499    0.038462   0.113761    0.067039
13          14       471500    0.035714   0.113761    0.067039
14          15       471501    0.000000   0.113761    0

In [13]:
#create final training_data file -merge promoted_content and df_events
gc.collect()
X=events.merge(promoted_content,how='left',on='ad_id')
promoted_content=promoted_content.drop('document_id',1)
df=events.drop(['ad_id','display_id','clicked'],1)
del events
df=df.drop_duplicates()

X (join of pl_score and promoted_content) creation done


In [14]:
Y=np.array(X['clicked'])
X= X.drop(['ad_id','display_id','clicked','document_id','ad_id_pl'],1)
X=np.array (X)
print("X array created")

X array created


In [16]:
print(X)
print(Y)
print(df)

[[ 0.19629527  0.16701992  0.2225146   0.16701992]
 [ 0.1689008   0.16324856  0.170883    0.15096548]
 [ 0.29403468  0.32420757  0.31435508  0.31435508]
 ..., 
 [ 0.37459283  0.37459283  0.21405144  0.35585415]
 [ 0.24398178  0.24398178  0.18644197  0.1679962 ]
 [ 0.17469011  0.16773028  0.14241466  0.14241466]]
[0 0 1 ..., 0 0 0]
          ad_id_pl  ad_pl_Score
0          42337_3     0.196295
1         139684_3     0.168901
2         144739_3     0.294035
3         156824_3     0.039135
4         279295_3     0.264000
5         296965_3     0.000000
6         125211_2     0.150410
7         156535_2     0.000000
8         169564_2     0.094862
9         308455_2     0.352018
10         71547_2     0.080171
11         95814_2     0.235476
12        152141_2     0.163235
13        183846_2     0.202232
14        228657_2     0.154098
15        250082_2     0.038334
16        149930_2     0.146012
17        153623_2     0.233444
18        184709_2     0.069848
19        186849_2     0.14

In [15]:
click_train=click_train.merge(df,on='ad_id_pl' ,how='left')
del df
gc.collect()
click_train=click_train.merge(promoted_content,how='left',on='ad_id')
del promoted_content
click_train= click_train.drop(['display_id','ad_id','platform','ad_id_pl'],1)
click_train.fillna(value=0,inplace=True)
print("X_test created")

X_test created


In [ ]:
from sklearn.svm import SVC
SVC(C=1.0, class_weight=None, coef0=0.0,
    decision_function_shape=None, degree=2, gamma='auto', kernel='rbf',
    max_iter=-1, probability=True, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
clf=SVC()
clf.fit(X,Y)
print(clf.score(X,Y))

In [15]:
from sklearn.naive_bayes import GaussianNB
gnb_clf = GaussianNB()
gnb_clf.fit(X, Y)
print(accuracy_score(Y, gnb_clf.predict(X)))
print(gnb_clf.score(X,Y))

0.769427199008
0.769427199008


In [17]:
#Applying Model
gc.collect()
reg=SGDClassifier(loss= 'log', penalty= 'l2')
reg.fit(X,Y)
print(accuracy_score(Y, reg.predict(X)))
print(reg.score(X,Y))
output= reg.decision_function(X_test)
#plt.hist(output)
#plt.show()
print("model fit done")

0.809769236739
0.809769236739
model fit done


In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state=1, class_weight="balanced", n_estimators=25, max_depth=6)
rf.fit(X, Y)
print(accuracy_score(Y, rf.predict(X)))
print(rf.score(X,Y))

In [20]:
#Create submission File
output=pd.read_csv('/home/nisha/Outbrain/clicks_test.csv').merge(pd.DataFrame(output,columns=['prediction']), left_index=True,right_index=True)
output.sort_values(['display_id','prediction'], inplace=True, ascending=False)
#.isin(16874594,16874595,16874596,16874597,16874598)].sort('prob_Score'))
subm = output.groupby('display_id').ad_id.apply(lambda x: " ".join(map(str,x))).reset_index()
print(subm[subm["display_id"]==16874594])
#subm.to_csv('/home/nisha/Outbrain/submission_file_1.csv', index=False)
#print("output stored")
print(type(subm))

   display_id                                     ad_id
0    16874594  170392 172888 162754 66758 150083 180797
<class 'pandas.core.frame.DataFrame'>


In [23]:
subm.to_csv(r'submission_file_2.csv', index=False)
print("done")

done
